# Big Data Analytics: NYC Crashes

## Einleitung

### Rahmenbedingungen des Projekts

### Aufbau der Datenpipeline

#### Installation der benötigten Docker-Container

### Datenquelle

### Analyseziele

### Installation der Python Packages

In [ ]:
pip install kafka-python pymongo

### Importieren benötigter Module

In [1]:
from kafka import KafkaProducer, KafkaConsumer
from pymongo import MongoClient
import datetime as dt
import requests
import os
from pathlib import Path

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Herunterladen der Datensätze (H)
Zum herunterladen der Datensätze werden die Daten von der Datenquelle mithilfe des nachfolgenden Python-Scripts heruntergeladen. Resultat sind drei `.csv`-Dateien die die Unfalldaten zeilenweise enthalten.

In [9]:
if not os.path.exists('data/'):
    os.mkdir('data')

for file_name, download_url in [
    ('crashes.csv', 'https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv?accessType=DOWNLOAD'),
    ('vehicles.csv', 'https://data.cityofnewyork.us/api/views/bm4k-52h4/rows.csv?accessType=DOWNLOAD'),
    ('persons.csv', 'https://data.cityofnewyork.us/api/views/f55k-p6yu/rows.csv?accessType=DOWNLOAD'),
]:
    if not os.path.isfile(fp:= (Path('data') / file_name)):
        with open(fp, 'wb') as crash_file:
            crash_file.write(requests.get(download_url).content)

In [ ]:



producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

dataset = open('Motor_Vehicle_Collisions_-_Crashes.csv', encoding='utf-8')
rows = dataset.readlines()[1:]

for i, row in enumerate(rows):
        producer.send('nyc_crashes', value=bytearray(row, encoding='utf-8'), key=bytearray(str(i), encoding='utf-8'))

consumer = KafkaConsumer('nyc_crashes', bootstrap_servers=['localhost:9092'], auto_offset_reset="earliest")


for msg in consumer:
    print(msg.value.decode('utf-8'))

In [ ]:


producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

for i in range(1, 13):
    if i < 10:
        i = "0%s" % i
    dataset = open('./data/2019/yellow-cabs-2019-%s.csv' % i, encoding='utf-8')
    rows = dataset.readlines()[1:]

    for i, row in enumerate(rows):
        producer.send('yellow-cabs', value=bytearray(row, encoding='utf-8'), key=bytearray(str(i), encoding='utf-8'))

for i in range(1, 7):
    if i < 10:
        i = "0%s" % i
    dataset = open('./data/2019/yellow-cabs-2020-%s.csv' % i, encoding='utf-8')
    rows = dataset.readlines()[1:]

    for i, row in enumerate(rows):
        producer.send('yellow-cabs', value=bytearray(row, encoding='utf-8'), key=bytearray(str(i), encoding='utf-8'))

In [ ]:


consumer = KafkaConsumer('yellow-cabs', bootstrap_servers=[""])

client = MongoClient("")

yellow_collection = client['datawarehouse']['bg-yellowcabs']
yellow_collection.delete_many({})

count = 0

for msg in consumer: 
    count += 1
    print('Received new message: %s' % count)
    values = msg.value.decode('utf-8').split(',')
    
    yellow_collection.insert_one({
        'pickup_datetime': dt.datetime.strptime(values[1], "%Y-%m-%d %H:%M:%S"),
        'dropoff_datetime': dt.datetime.strptime(values[2], "%Y-%m-%d %H:%M:%S"),
        'passenger_count': int(values[3]),
        'trip_distance': float(values[4]),
        'PULocationID': values[5],
        'DOLocationID': values[6],
        'payment_type': int(values[9]),
        'fare_amount': float(values[10]),
        'tip_amount': float(values[15]),
        'total_amount': float(values[16])
    })